<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Advanced Data Science<br/>Natural Language Processing</center>

# S5 Challenge. Word embeddings puestos a prueba

# Encontrar preguntas similares en StackOverflow

En este challenge vamos a calcular la similaridad entre dos trozos documentos. Utilizando esta técnica vamos a aprender a buscar preguntas duplicadas en [StackOverflow](https://stackoverflow.com).

### Librerías

- [Gensim](https://radimrehurek.com/gensim/)
- [Numpy](http://www.numpy.org) 
- [scikit-learn](http://scikit-learn.org/stable/index.html) 
- [Nltk](http://www.nltk.org)

In [1]:
import utils_05
import gensim
import numpy as np
from typing import List, Tuple
from gensim.models import KeyedVectors,Word2Vec
import math
import nltk
import re
from nltk.corpus import stopwords

### Datos

Lo primero es descargar los datos necesarios, que se guardarán en `week3/data`.

In [2]:
from download_utils import download_embeddings_challenge
download_embeddings_challenge()

File already downloaded!
Done!


## Word embedding

En la tarea vamos a usar word-embeddings, tanto preentrenados como entrenados por nosotros de cero:

 - [Word vectors Pre-entrenados](https://code.google.com/archive/p/word2vec/) de Google que fueron entrenados en una parte del dataset 'Google News' (con alreadedor de 100K millones de palabras). Es un modelo con vectores de 300 dimensiones, y un vocabulario de 3 millones de palabras. Están en el archivo `GoogleNews-vectors-negative300.bin.gz`.
 - Extra: Vamos a entrenar un modelo en nuestros datos usando gensim.

Siempre es más fácil comenzar con embeddings pre-entrenados. Carga los vectores de Goggle pre-entrenados y cárguelos usando la función [gensim.models.KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) de gensim con el parámetro `binary = True`. Si el tamaño de los vectores es mayor que la memoria disponible, puede cargar solo una parte del vocabulario definiendo el parámetro `limit` (recomendado: 500000).

In [3]:
path = '../embeddings/GoogleNews-vectors-negative300.bin'
### ESCRIBE AQUÍ TU CÓDIGO
wv_embeddings = KeyedVectors.load_word2vec_format(path, binary=True)  # Cargamos el formato binario
###

### ¿Cómo utilizar los embeddings ?

Una vez que haya cargado las representaciones, asegúrate de poder acceder a ellas. Primero, puedes verificar si los embeddings cargados contienen una palabra:

    'word' in wv_embeddings
    
También, puedes obtener el embedding correspondiente accediendo como si fuera un diccionario:

    wv_embeddings['word']
 
### Check de los embeddings
 
Para evitar errores durante la primera parte, podemos verificar que los embeddings cargados son correctos. Llama a la función `check_embeddings()`, implementada a continuación, que ejecuta 3 pruebas:
1. Encuentra la palabra más similar para las palabras "positivas" y "negativas" proporcionadas.
2. Encuentra qué palabra de la lista dada no coincide con las demás.
3. Encuentra la palabra más similar a la dada.

En el caso correcto, la función devolverá la cadena *¡Todo correcto!*. De lo contrario, repasa el código anterior.

In [4]:
wv_embeddings['word']

array([ 3.59375000e-01,  4.15039062e-02,  9.03320312e-02,  5.46875000e-02,
       -1.47460938e-01,  4.76074219e-02, -8.49609375e-02, -2.04101562e-01,
        3.10546875e-01, -1.05590820e-02, -6.15234375e-02, -1.55273438e-01,
       -1.52343750e-01,  8.54492188e-02, -2.70996094e-02,  3.84765625e-01,
        4.78515625e-02,  2.58789062e-02,  4.49218750e-02, -2.79296875e-01,
        9.09423828e-03,  4.08203125e-01,  2.40234375e-01, -3.06640625e-01,
       -1.80664062e-01,  4.73632812e-02, -2.63671875e-01,  9.08203125e-02,
        1.37695312e-01, -7.20977783e-04,  2.67333984e-02,  1.92382812e-01,
       -2.29492188e-02,  9.70458984e-03, -7.37304688e-02,  4.29687500e-01,
       -7.93457031e-03,  1.06445312e-01,  2.80761719e-02, -2.29492188e-01,
       -1.91650391e-02, -2.36816406e-02,  3.51562500e-02,  1.71875000e-01,
       -1.12304688e-01,  6.25000000e-02, -1.69921875e-01,  1.29882812e-01,
       -1.54296875e-01,  1.58203125e-01, -7.76367188e-02,  1.78710938e-01,
       -1.72851562e-01,  

In [5]:
print(utils_05.check_embeddings(wv_embeddings))

¡Todo correcto!


## Convierte el texto a word embeddings

### **Tarea 1 (Question2Vec):**

Por lo general, se usan embeddings para las palabras, pero para la tarea necesitamos crear una representación para toda la pregunta. Se podría hacer de diferentes maneras. En nuestro caso, usaremos una **media** de todos los vectores de palabras en la pregunta. Para ello hay que implementar la función `question_to_vec()`, que calcula la representación de preguntas descrita anteriormente. Esta función debería funcionar con el texto de entrada tal como está, sin ningún procesamiento previo.

Ten en cuenta que podría haber palabras sin un embedding. En este caso, no tengas en cuenta esas palabras para calcular el embedding del texto. Si la pregunta no tiene ninguna palabra conocida, la función debería devolver un vector de ceros (pero del mismo tamaño que el resto).

In [6]:
def question_to_vec(
    question: str, embeddings: gensim.models.keyedvectors.KeyedVectors, dim: int = 300
):
    """
    question: a string
    embeddings: dict where the key is a word and a value is its' embedding
    dim: size of the representation

    result: vector representation for the question
    """
    ######################################
    ### ESCRIBE AQUÍ TÚ CÓDIGO
    ######################################
    # recoremos el texto, sacamos los tokens. luego su vector y lueho el promedio de las palabras-
    
    v_result = np.zeros(dim)
    tokens = question.split()
#     tokens = nltk.tokenize.word_tokenize(question)
    i_token_count = 0
    for token in tokens:
        
        try:
            v_emmbedding = embeddings[token]
        except KeyError:   
             #print(f"No existe {token} en el embedding.")
            pass

        else:
            i_token_count+=1
            v_result=v_result+v_emmbedding
            
    if i_token_count:
        v_result=v_result/i_token_count
    
    result = v_result
#     print(result)
    return result


Para comprobar que la implementación es correcta, corre `question_to_vec_tests`.

In [7]:
print(utils_05.question_to_vec_tests(question_to_vec, wv_embeddings))

Test correctos.


## Evaluación de similitud de texto. Problemas de ranking

Ahora tenemos un método para crear una representación de cualquier frase y esto nos da una forma de resolver el problema. Primero, necesitamos comprobar qué tan bien funciona nuestra solución (los vectores de Google + `question_to_vec ()`), y para ello lo primero que tenemos que determinar **es cómo evaluar la calidad de un modelo**.

Podemos esperar que, si usamos buenos *embeddings*, la **similitud** entre los vectores de las preguntas duplicadas debería ser menor que entre dos aleatorias. Ordenando según la **similitud del coseno**, para cada sentencia podemos ordenar el resto de sentencias de la base de datos, y comprobar en qué posición queda la pregunta duplicada. Podemos asumir que el resto de preguntas **no son parecidas**, es decir, son casos negativos.


Por ejemplo, si tenemos la pregunta _"Exceptions What really happens"_ y estamos seguros de que otra pregunta _"How does the catch keyword determine the type of exception that was thrown"_ es un duplicado. Pero nuestro modelo no lo sabe e intenta encontrar la mejor opción también entre preguntas como _"How Can I Make These Links Rotate in PHP"_ , _"NSLog array description not memory address"_ y _"PECL_HTTP not recognised php ubuntu"_. El objetivo del modelo es ordenar todas estas 4 preguntas (1 *positiva* y *R* = 3 *negativas*) de tal forma que la duplicada esté en la primera posición.



En general, lo normal no es que el mejor candidato esté en la primera posición, y no queremos una métrica tan rígica que sólo considere una respuesta como buena si el resultado correcto está en la primera posición. Se pueden utilizar métricas **más suaves**, en las que tomamos la posición en la que se encuentra el candidato.

Supongamos que nos fijamos en los primeros $K$ elementos del ranking propuestos por el modelo, y $N$ es el número total de documentos que son duplicados, es decir, los **casos positivos** que hay en el dataset. Dos de las métricas más comunes son:


### Hits @ K

La primera métrica es simplemente el promedio de resultados correctos con *K* fijo:

$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

donde $q_i$ es la i-ésima consulta, $dup_i$ es su duplicado, $topK(q_i)$ son las $K$ primeras propuestas proporcionadas por nuestro modelo y el operador **indicador** $[dup_i \in topK(q_i)]$ es igual a 1 si el duplicado $q_i$ está entre las primeras $K$ propuestas y 0 en caso contrario (para más detalles sobre el operador [aquí](https://en.wikipedia.org/wiki/Iverson_bracket).


### DCG @ K
La segunda propuesta tiene más en cuenta la posición del ranking de la respuesta correcta. Es una [métrica DCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) simplificada:

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

donde $rank_{dup_i}$ es una posición del duplicado en el ranking para la consulta $ q_i $. Según esta métrica, el modelo obtiene una mayor recompensa por una posición más alta en el ranking. Si la respuesta no aparece en topK, la recompensa es cero.

### Ejemplo de las métricas

Vamos a calcular las métricas definidas sobre el ejemplo que hemos introducido antes. En este caso tenemos que $N$ = 1 (sólo hay un duplicado), y que lo correcto es que $q_1$ fuera _"How does the catch keyword determine the type of exception that was thrown"_. Vamos a imaginar que el modelo devuelve el siguiente ranking:
1. _"How Can I Make These Links Rotate in PHP"_
2. _"How does the catch keyword determine the type of exception that was thrown"_
3. _"NSLog array description not memory address"_
4. _"PECL_HTTP not recognised php ubuntu"_

Usando este ranking, calcula la métrica _Hits@K_ para $K$ = 1, 2, 4: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ porque la respuesta correcta no aparece en el _top1_.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ porque $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Usando este ranking, calcula la métrica _DCG@K_ para $K$ = 1, 2, 4:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ porque la respuesta correcta no aparece en el _top1_.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, porque $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


### **Tasks 2 y 3 (HitsCount y DCGScore):** 

Implementa las funciones `hits_count()` y `dcg_score()`, como se han descrito antes. Cada función tiene dos entradas, `dup_ranks` and `k`. Donde `dup_ranks` es la lista que contiene las posiciones en el ranking de los duplicados. Por ejemplo, en el ejemplo anterior la longitud de `dup_ranks` es $N$=1, y `dup_ranks = [2]`, puesto que la respuesta correcta está en la posición 2.

In [8]:
def hits_count(dup_ranks: List[int], k: int) -> float:
    """
    dup_ranks: list of duplicates' ranks; one rank per question;
               length is a number of questions which we are looking for duplicates;
               rank is a number from 1 to len(candidates of the question);
               e.g. [2, 3] means that the first duplicate has the rank 2, and the second 3.
    k: number of top-ranked elements (k in Hits@k metric)

    result: return Hits@k value for current ranking
    """
    ######################################
    ### ESCRIBE AQUÍ TÚ CÓDIGO
    ######################################
    
    dup_ranks_ = np.asarray(dup_ranks)
    k_ = np.array([k]) # Convertimos K en numpy array para poder operar de forma más optima.
    N=len(dup_ranks_) # Númeero de elementos repetidos
    result = np.sum(dup_ranks_ <= k_)/N # Generamos la media de las veces que aparecen dentro del ranking
    return result

Vamos a comprobar que está bien programado:

In [9]:
print(utils_05.test_hits(hits_count))

Tests correctos.


In [10]:
def dcg_score(dup_ranks: List[int], k: int) -> float:
    """
    dup_ranks: list of duplicates' ranks; one rank per question;
               length is a number of questions which we are looking for duplicates;
               rank is a number from 1 to len(candidates of the question);
               e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
    k: number of top-ranked elements (k in DCG@k metric)

    result: return DCG@k value for current ranking
    """
    ######################################
    ### ESCRIBE AQUÍ TÚ CÓDIGO
    ######################################
    dup_ranks_ = np.asarray(dup_ranks)
    k_ = np.array([k])
    N=len(dup_ranks_) # Número de elementos repetidos
    
    # Se incrementará de (1/log2(1+ranking)) en (1/log2(1+ranking)) cada vez 
    #que el ranking del duplicado este dentro del top K
    result = np.sum( (1./(np.log2(1. + dup_ranks_)))*(dup_ranks_ <= k_) )/N
    #     print(result)
    
    return result

In [11]:
print(utils_05.test_dcg(dcg_score))

Tests correctos.


##  Algoritmo: embeddings pre-entrenados

Tenemos sólo un corpus: **test**. Cada entrada separada por '\t', y el formato es el siguiente:
 - *test* tiene las siguientes columnas: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 

Vamos a leer el corpus de *test*, en `data/test_small.tsv`. Que usaremos para evaluar nuestra solución.

In [12]:
def read_corpus(filename: str) -> List[List[str]]:
    data = []
    for line in open(filename, encoding="utf-8"):
        data.append(line.strip().split("\t"))
    return data

In [13]:
test = read_corpus('data/test_small.tsv')

In [14]:
len(test)

1500

In [15]:
test[0]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?',
 'How do I declare a C array in Swift?',
 'Using rand() when flipping a coin and rolling a die',
 'Handling a JSON field with a special character in its name in Java',
 'React Native select row on ListView when push it',
 "Get 'creation_time' of video using ffmpeg and regex",
 'Does row exist and multiple where',
 "How to specify a classifier in a gradle dependency's dependency?",
 'Using $unwind on multiple do

In [16]:
len(test[0])

1001

### **Tarea 4 (ranking)**

Ahora que tenemos como construir las *features*, y tenémos métricas para evaluar como de bueno es nuestro algoritmo de ranking, vamos a construir la función que nos hace el ranking.

Para comparar preguntas, usaremos la similaridad de coseno o *cosine similarity*. Tenemos que implementar la función `rank_candidates()`. La función debe retornar una lista ordenada de pares *(índice inicial en la lista de candidatos, candidato)*.

Por ejemplo, si la lista de candidatos es `[a,b,c]` y el más similar es `c`, después `a`, y luego `b`, entonces la función debe retornar `[(2, c), (0, a), (1, b)]`

**Ayuda:** Puedes utilizar la función `cosine_similarity()` de `sklearn.metrics.pairwise`. 

En cualquier caso, es preferible usar una versión vectorizada de la función `cosine_similarity()`, es decir, que no utilice bucles ni *list comprehension*. Si no el cómputo será demasiado lento. La función de sklearn cumple esto.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity


def rank_candidates(
    question: str,
    candidates: List[str],
    embeddings: gensim.models.keyedvectors.KeyedVectors,
    dim: int = 300,
) -> List[Tuple[int, str]]:
    """
    question: a string
    candidates: a list of strings (candidates) which we want to rank
    embeddings: some embeddings
    dim: dimension of the current embeddings

    result: a list of pairs (initial position in the list, question)
    """

    ######################################
    ### ESCRIBE AQUÍ TÚ CÓDIGO
    ######################################
    #Obtenemos el vector de representación de la quetion
    v_repre_question=np.asarray([question_to_vec(question,embeddings,dim),])
    #Obtenemos los vectores de representación de los candidatos
    l_v_repre_candidates = [question_to_vec(candidate,embeddings,dim) for candidate in candidates]
    a_v_repre_candidates =  np.asarray(l_v_repre_candidates)
    
    # Calculamos la simlitud del coseno entre el vector de la question y los vectores de los candidatos.
    v_result_cosine = cosine_similarity(v_repre_question,a_v_repre_candidates)
#     print (f'Cosine_similarity : {result_cosine}')
    
    # Ordenamos los indices de los valores obtenidos de mayor a menor similitud
    v_result_cosine = np.argsort(v_result_cosine[0])[::-1]
#     print(result_cosine)
    
    #Construimos la lista de tuplas
    result=[(index, candidates[index]) for index in v_result_cosine]
#     print(result)
    return result

Comprobamos que es correcto:

In [18]:
print(utils_05.test_rank_candidates(rank_candidates, wv_embeddings))

Test correctos.


Ahora comprobamos cómo de bueno es nuestro algoritmo. Ejecuta las siguientes celdas para obtener los resultados.

El cálculo de la similaridad entre vectores lleva su tiempo, y este cálculo puede llevar unos 2 minutos.

In [19]:
wv_ranking = []
for line in test:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [20]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.211 | Hits@   1: 0.211
DCG@   5: 0.268 | Hits@   5: 0.319
DCG@  10: 0.285 | Hits@  10: 0.370
DCG@ 100: 0.324 | Hits@ 100: 0.567
DCG@ 500: 0.357 | Hits@ 500: 0.825
DCG@1000: 0.376 | Hits@1000: 1.000


Si has hecho todos los pasos correctamente, debes estar un poco frustrado con los resultados obtenidos. Vamos a intentar entender por qué la calidad es tan baja.

Lo primero, antes de trabajar con los datos, hay que explorarlos para tener una idea de cómo son. Vamos a imprimir algunas de las preguntas del dataset:

In [21]:
for line in test[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


Como puedes ver, estamos tratando con datos sin procesar. Tienen singos de puntuación, caracteres especiales y letras en mayúsculas. Esto provoca que para muchas palabras no tengamos un *word embedding* correspondiente. Por ejemplo, para la palabra "grid?". 

Para resolver esto, vamos a utilizar la función `text_prepare()`, que preprocesa el texto. Puedes usar también cualquiera de las usadas en los challenges anteriores.

Ahora, prepara el corpus de test:

In [22]:
from utils_05 import text_prepare

prepared_test = []
for line in test:
    prepared_test.append([text_prepare(sentence) for sentence in line])

Ahora evaluamos los resultados sobre los datos preprocesados:

In [23]:
wv_prepared_ranking = []
for line in prepared_test:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [24]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

DCG@   1: 0.336 | Hits@   1: 0.336
DCG@   5: 0.399 | Hits@   5: 0.454
DCG@  10: 0.417 | Hits@  10: 0.511
DCG@ 100: 0.449 | Hits@ 100: 0.670
DCG@ 500: 0.471 | Hits@ 500: 0.841
DCG@1000: 0.488 | Hits@1000: 1.000


## Extra: Usa un modelo de fasttext en lugar de word2vec

Para tener información sobre los modelos pre-entrenados disponibles, visita la siguiente web.

https://fasttext.cc/docs/en/crawl-vectors.html

Con la librería de Python `fasttext` podrás descargarlos de forma sencilla.

**Imprime las métricas DCG@K y Hits@K al igual que en los modelos anteriores**

In [25]:
%%timeit
# descargamos un modelo, lleva bastante tiempo
# url = "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz"
import sys
!{sys.executable} -m pip install fasttext

import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English

You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


4.81 s ± 40.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


You should consider upgrading via the 'C:\Users\JPAS\Anaconda3\envs\mioti_v2\python.exe -m pip install --upgrade pip' command.


In [26]:
# cargamos el modelo tipo fasttext en gensim
# la ventaja de cargar el modelo y no los vectores es que tenemos OOV (out-of-vocabulary) vectors
ft_model = gensim.models.fasttext.load_facebook_model('cc.en.300.bin.gz')
ft_embeddings = ft_model.wv

In [27]:
ft_embeddings['world']

array([-4.86496761e-02, -4.55158651e-02, -9.72260237e-02,  4.03840579e-02,
       -3.90644334e-02, -7.71286935e-02,  8.89645442e-02,  3.61033492e-02,
        1.11490022e-03, -8.33936967e-04, -7.75447115e-03,  9.72571597e-02,
       -1.11513743e-02, -1.02696680e-02, -5.11458106e-02,  1.87770724e-02,
       -1.34799145e-02, -8.55755731e-02, -3.66249420e-02,  2.39651948e-02,
        3.84481214e-02,  2.85097957e-03, -1.21261068e-02,  4.35700491e-02,
       -7.63629824e-02, -2.72414945e-02,  9.94105823e-03, -4.59332839e-02,
       -5.97596243e-02,  5.27391434e-02, -1.35981468e-02, -8.02709684e-02,
        2.62324177e-02, -6.37386590e-02,  5.42014651e-03, -7.44198859e-02,
       -5.91174401e-02,  6.13477156e-02,  2.42813528e-02, -2.03960761e-02,
       -7.01110363e-02, -3.63789126e-03, -3.30209360e-02, -8.02930072e-03,
       -7.72294253e-02, -2.17669457e-02, -1.08852968e-01, -6.58725128e-02,
        3.80129702e-02,  1.28018633e-02,  1.88515075e-02,  1.51783484e-03,
        2.71031242e-02, -

In [28]:
######################################
### ESCRIBE AQUÍ TÚ CÓDIGO
######################################

from utils_05 import text_prepare

prepared_test = []
for line in test:
    prepared_test.append([text_prepare(sentence) for sentence in line])
    
ft_wv_prepared_ranking = []
for line in prepared_test:
    q, *ex = line
    ranks = rank_candidates(q, ex, ft_embeddings)
    ft_wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)
    

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ft_wv_prepared_ranking, k), 
                                              k, hits_count(ft_wv_prepared_ranking, k)))   

DCG@   1: 0.305 | Hits@   1: 0.305
DCG@   5: 0.369 | Hits@   5: 0.429
DCG@  10: 0.388 | Hits@  10: 0.487
DCG@ 100: 0.431 | Hits@ 100: 0.697
DCG@ 500: 0.456 | Hits@ 500: 0.893
DCG@1000: 0.468 | Hits@1000: 1.000


## Extra: Entrena tu propio word-embedding con gensim y usálo para crear un algoritmo de ranking

* El primer word-embedding que vamos a entrenar lo vamos ha hacer utilizando el libro del Quijote. Aplicaremos el tratamiento correspondiente y después probaremos un ranking con frases de las novelas ejemplares de Cervantes

In [29]:
import requests
r = requests.get("https://www.gutenberg.org/cache/epub/2000/pg2000.txt")
with open("don_quijote.txt", "wb") as f:
    f.write(r.content)
r.close()
test = read_corpus('don_quijote.txt')

In [30]:
for line in test[-10:]:
    q, *examples = line
    print(q, *examples)



Most people start at our Web site which has the main PG search facility:

https://www.gutenberg.org

This Web site includes information about Project Gutenberg-tm,
including how to make donations to the Project Gutenberg Literary
Archive Foundation, how to help produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.


In [31]:
import re
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
STOPWORDS_ES = set(stopwords.words('spanish'))

def text_prepare_es_en(text):
    sentences = []
    for sentence in text:
            for line in sentence:

                line = line.lower()
                line = REPLACE_BY_SPACE_RE.sub(' ', line)
                line = GOOD_SYMBOLS_RE.sub('', line)
                line = ' '.join([x for x in line.split() if x and x not in STOPWORDS_ES])
                line = ' '.join([x for x in line.split() if x and x not in STOPWORDS])
                line = line.strip()
                line = str.split(line)
                if(len(line)>0):
                    sentences.append(line)
    return sentences


In [32]:
# define los datos de entrenamiento
sentences=text_prepare_es_en(test)

In [33]:
sentences[250:257]

[['seguistes', 'yndole', 'ello', 'cuanto', 'ms', 'si', 'bien', 'caigo'],
 ['cuenta', 'libro', 'necesidad', 'ninguna', 'cosa'],
 ['aquellas', 'vos', 'decs', 'falta', 'l', 'invectiva'],
 ['libros', 'caballeras', 'nunca', 'acord', 'aristteles', 'dijo'],
 ['san', 'basilio', 'alcanz', 'cicern', 'caen', 'debajo', 'cuenta'],
 ['fabulosos', 'disparates', 'puntualidades', 'verdad', 'observaciones'],
 ['astrologa', 'importancia', 'medidas', 'geomtricas']]

In [34]:
from gensim.models import Word2Vec

# entrenamiento modelo
model = Word2Vec(sentences, min_count=1,vector_size=300)
# summarize el modelo cargado
print(model)
# summarize vocabulario
print(model.wv)
#salvamos el modelo    
model.save('my_model.bin')

# cargamos el modelo
new_model = Word2Vec.load('my_model.bin')
my_wv_embeddings = new_model.wv
print(new_model)
my_wv_embeddings['sancho']

Word2Vec(vocab=22765, vector_size=300, alpha=0.025)
Word2Vec(vocab=22765, vector_size=300, alpha=0.025)


array([-9.04883295e-02,  6.98136270e-01,  2.71197915e-01,  2.28709295e-01,
        5.10908403e-02, -8.45430553e-01,  6.18608892e-01,  1.36443603e+00,
        1.76739872e-01, -2.96765387e-01,  2.70668954e-01, -2.51963824e-01,
        2.00709596e-01,  1.21149220e-01, -2.71378815e-01, -4.21786875e-01,
        3.95062894e-01, -1.61175117e-01,  9.09386650e-02, -1.83225781e-01,
       -2.60947317e-01, -2.69418340e-02,  4.80208009e-01,  2.76008010e-01,
        4.43885803e-01,  9.66602564e-02, -6.83615923e-01,  7.32327327e-02,
       -3.17467421e-01, -5.44813991e-01,  1.38510853e-01, -4.00048792e-01,
        5.40683120e-02, -2.56613225e-01, -9.21578184e-02,  2.47084230e-01,
        8.23650286e-02, -4.28261578e-01, -2.36538872e-01, -1.10327065e-01,
       -5.07121205e-01,  6.73128515e-02,  1.30832255e-01, -3.44049931e-01,
        3.00084531e-01,  3.83489549e-01,  9.80851799e-02,  1.13953762e-01,
       -2.81675637e-01,  5.65839648e-01,  1.42017901e-01, -2.50716005e-02,
       -4.62955475e-01,  

In [35]:
#Se definen los candidatos

question='Confía en el tiempo, que suele dar dulces salidas a muchas amargas dificultades'
candidates=['Nunca el consejo del pobre, por bueno que sea, es admitido','Yo no soy bueno para palacio, porque tengo vergüenza y no sé lisonjear',
           'En la lengua consisten los mayores daños de la vida humana','Las honestas palabras dan indicio de la honestidad del que las pronuncia o las escribe',
           'Cuando supe tu desgracia, me pesó en el alma, y se aseguró mi pecho, que estaba sobresaltado','Aún entre los demonios hay unos peores que otros, y entre muchos malos hombres suele haber alguno bueno']

In [36]:
#Se imprimen los ranking de la frase y la pregunta
rank_candidates(question, candidates, my_wv_embeddings)

[(3,
  'Las honestas palabras dan indicio de la honestidad del que las pronuncia o las escribe'),
 (5,
  'Aún entre los demonios hay unos peores que otros, y entre muchos malos hombres suele haber alguno bueno'),
 (2, 'En la lengua consisten los mayores daños de la vida humana'),
 (0, 'Nunca el consejo del pobre, por bueno que sea, es admitido'),
 (1, 'Yo no soy bueno para palacio, porque tengo vergüenza y no sé lisonjear'),
 (4,
  'Cuando supe tu desgracia, me pesó en el alma, y se aseguró mi pecho, que estaba sobresaltado')]

In [41]:
#Se imprimen las métricas DCG@K y Hits@K
from utils_05 import text_prepare

prepared_test = []
for line in candidates:
    prepared_test.append([text_prepare(sentence) for sentence in line])
    
my_wv_prepared_ranking = []
for line in prepared_test:
    q, *ex = line
    ranks = rank_candidates(q, ex, my_wv_embeddings)
    my_wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)
    

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(my_wv_prepared_ranking, k), 
                                              k, hits_count(my_wv_prepared_ranking, k)))   

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.000 | Hits@   5: 0.000
DCG@  10: 0.000 | Hits@  10: 0.000
DCG@ 100: 0.136 | Hits@ 100: 0.833
DCG@ 500: 0.161 | Hits@ 500: 1.000
DCG@1000: 0.161 | Hits@1000: 1.000


* En segundo lugar vamos a entrenar un word-embedding con una página de wikipedia sobre inteligencia artificial. Para ello primero tenemos que tener instaladas unas librerías para realizar correctamente la captura de internet

In [42]:
#Instalación de librerías

# !pip install beautifulsoup4
# !pip install lxml

In [43]:
#Se importan las librerías

import bs4 as bs
import urllib.request

In [44]:
#Se obtiene el texto de la página de wikipedia

scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence')
article = scrapped_data .read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

articulo = ""

for p in paragraphs:
    articulo += p.text

In [45]:
#Pasamos al preprocesamiento del texto

articulo_min = articulo.lower()
articulo_regex = re.sub('[^a-zA-Z]', ' ', articulo_min )
articulo_regex = re.sub(r's+', ' ', articulo_regex)

#tokenización

articulo_frases=nltk.sent_tokenize(articulo_regex)
articulo_palabras = [nltk.word_tokenize(frase) for frase in articulo_frases]

#Eliminación de stopwords

for i in range(len(articulo_palabras)):
    articulo_palabras[i] = [w for w in articulo_palabras[i] if w not in stopwords.words('english')]

In [46]:
#Se entrena el modelo

model=Word2Vec(articulo_palabras,min_count=1,vector_size=300)
print(model)

Word2Vec(vocab=1737, vector_size=300, alpha=0.025)


In [47]:
#Se visualiza la vectoización de una palabra

model.wv['intelligence']

array([-1.7147678e-03, -8.4864514e-05, -2.3740393e-03,  3.6334898e-03,
       -3.9545275e-04, -4.4211429e-03,  2.5741529e-04,  5.8221230e-03,
        6.1479048e-05, -2.2252637e-03,  5.7122915e-04, -2.6932163e-03,
       -2.0240669e-04,  4.5642816e-04, -2.4866257e-03,  1.2474941e-03,
        2.6945374e-03, -3.8034604e-03,  2.4679652e-03, -3.0559527e-03,
       -1.2827468e-03,  1.5120611e-03, -1.8172583e-04, -1.9035680e-04,
        4.3028640e-03, -1.2814786e-03, -4.9921996e-03,  2.7034350e-03,
       -1.8261098e-03, -2.9813736e-03,  3.6321583e-03,  7.7805447e-04,
        1.0148692e-03,  2.6778437e-03, -3.1795900e-03,  2.4634872e-03,
        1.5754655e-03, -5.6849082e-04,  8.5842394e-04,  1.0505862e-03,
        8.0099783e-04,  2.1635678e-03, -1.3639734e-03, -3.3757936e-03,
        8.6707430e-04,  2.9361870e-03, -3.4430466e-04,  2.4327433e-03,
        9.1209513e-04,  3.7630496e-03,  7.9355808e-04,  2.8716193e-03,
       -1.7108674e-03,  3.6309729e-03,  4.4061005e-04, -4.7684161e-04,
      

In [48]:
#Se crea una variable con el word_embedding entrenado

self_embedding=model.wv

In [49]:
#Vamos a crear una lista de frases candidatas a las que aplicar el ranking

question='what is artificial intelligence'
candidates=['Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence',
           'Self-driving cars','How Does Artificial Intelligence Work',"AI is the branch of computer science that aims to answer Turing's question in the affirmative",
           'Norvig and Russell focus particularly on rational agents that act to achieve the best outcome','algorithms enabled by constraints, exposed by representations that support models targeted at loops that tie thinking, perception and action together',
           'Perceiving the world directly means that reactive machines are designed to complete only a limited number of specialized duties']

In [50]:
#Se imprimen los rankings de las frases frente a la pregunta
rank_candidates(question, candidates, self_embedding)

[(0,
  'Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence'),
 (6,
  'Perceiving the world directly means that reactive machines are designed to complete only a limited number of specialized duties'),
 (5,
  'algorithms enabled by constraints, exposed by representations that support models targeted at loops that tie thinking, perception and action together'),
 (3,
  "AI is the branch of computer science that aims to answer Turing's question in the affirmative"),
 (4,
  'Norvig and Russell focus particularly on rational agents that act to achieve the best outcome'),
 (2, 'How Does Artificial Intelligence Work'),
 (1, 'Self-driving cars')]

In [51]:
#Se imprimen las métricas DCG@K y Hits@K
from utils_05 import text_prepare

prepared_test = []
for line in candidates:
    prepared_test.append([text_prepare(sentence) for sentence in line])
    
wv_prepared_ranking = []
for line in prepared_test:
    q, *ex = line
    ranks = rank_candidates(q, ex, self_embedding)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)
    

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))   

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 0.000 | Hits@   5: 0.000
DCG@  10: 0.000 | Hits@  10: 0.000
DCG@ 100: 0.139 | Hits@ 100: 0.714
DCG@ 500: 0.177 | Hits@ 500: 1.000
DCG@1000: 0.177 | Hits@1000: 1.000
